In [1]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import re
import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import scipy as sp

In [2]:
df = pd.read_excel("特許リスト(21.12.6).xlsx",header=0) # engineは環境によってエラー出たり出なかったりだから、必要に応じて変える


In [3]:
df['詳細な説明'] = df['詳細な説明'].fillna(' ') #nanを文字列に
df['全請求項'] = df['全請求項'].fillna(' ')
df["company"] = [set(i.split(",")) for i in df['出願人・権利者名']]    #出願名をスプリットしてset型に
df['datetime'] = [dt.strptime(i.split(',')[0],  '%Y.%m.%d') for i in df['公開・公表日']]  #公開・公表日を datetime64[ns] 型（ Timestamp 型）に 、複数あるのは先頭のだけに
# df['sentence'] = df['発明の名称'] +df['要約'] + df['全請求項'] .astype("str") + df['詳細な説明']  #なぜか .astype("str") いれないとエラー
# df['sentence'] = df['発明の名称'] +df['要約'] + df['詳細な説明']  
# df['sentence'] = df['発明の名称'] +df['要約']        #sentenceの対象にする項目も必要に応じて変える
df['sentence'] = df['詳細な説明']        #sentenceの対象にする項目も必要に応じて変える

df = df.drop(columns=['早期請求出願有無', '要約', '全請求項', '詳細な説明', 'ST', 'ステイタス', '原出願日', '出願日', '公開・公表日', '出願日', '登録番号', '公開・公表番号', '出願人・権利者名'])

df = df.rename(columns={'発明の名称': 'patent_name', '筆頭ＩＰＣ': 'headIPC', '出願番号': 'id', 'Ｆターム': 'f_term', 'company': 'createdBy', 'datetime':'createdAt', 'terms':'term', '発明者名':'inventor', '筆頭発明者':'head_inventor', })

In [4]:
df.head(3)

,patent_name,id,head_inventor,inventor,IPC,headIPC,f_term,FI,createdBy,createdAt,sentence
0,Ｚ形仕切付段ボール箱,実願2001-003729,佐藤 浩康,佐藤 浩康,B65D 5/491 (2006.01),B65D 5/491 (2006.01),"3E060AA03,3E060AB05,3E060BC02,3E060CB04,3E060C...","B65D 5/48 101J,B65D 5/498",{佐藤工業株式会社},2001-12-14,【考案の詳細な説明】 【０００１】 【考案の属する技術分野】 本考案は、袋菓子または詰...
1,吊りフック,実願2001-002834,石塚 浩次,"石塚 浩次,織田 悦芳,好田 勝之",B66C 1/36 (2006.01),B66C 1/36 (2006.01),"3F004CC01,3F004CD08",B66C 1/36 A,"{東亜建設工業株式会社, 社団法人日本海上起重技術協会, 寄神建設株式会社}",2001-11-16,【考案の詳細な説明】【０００１】【考案の属する技術分野】 本考案は、クレーン等によりコンクリ...
2,橋脚柱の構築工法,特願2001-120638,大内 一,"大内 一,田中 浩一","E01D 19/02 (2006.01),E01D 21/00 (2006.01)",E01D 19/02 (2006.01),"2D059AA03,2D059CC04","E01D 19/02 ,E01D 21/04 ,E01D 21/00...",{株式会社大林組},2001-11-09,【発明の詳細な説明】【０００１】【産業上の利用分野】この発明は、橋脚柱の構築工法および橋脚柱...


In [10]:
excepword = ["位置","部","上記","上部","上端","下記","下部","下端","両端","内部","外部","前記","複数","横","図","要約","方法","手段","対象","現場","評価" ,"可能","構成","当該","以下","以上","該当","項","各","基準","端部","請求", "特許","本","一定","同上", '方向', '程度']  #除外する特定の名詞
exceptype= ["接尾","副詞可能","数詞","助数詞","代名詞","複数"]  #除外する名詞のタイプ


# 
def tokenize_rm_setsubi(sentence):
    """ 連名詞のリストをトークンとして返す """
#     print(".",end='') #動いてるのか不安になるからプリント
    sentence = re.sub('\d+', '', sentence)   # 数字を削除(macabでなぜか名詞に認定された数字あったからここで)
    sentence = re.sub(r'[ -/:-@\[-~]', "。", sentence)#半角記号,数字,英字
    sentence = re.sub(r'[！-／：-＠［-｀｛-～、-〜”’%]+/g', "。", sentence)#全角記号
    # sentence = re.sub(r'[[０-９ａ-ｚＡ-Ｚ]]', '', sentence) #全角英数を削除
    
    
    wakati = MeCab.Tagger("")
    node = wakati.parseToNode(sentence)
    sequence = 2   #何連名詞以上をかえすか
    dictionary = []
    prev_seq = 0  #その前にいくつ名詞続いたか
    flag = 0  #
    while node:
        word = node.surface
        hinshi = node.feature.split(",")

        if prev_seq  : # その前に名詞が1以上つづいていたら
            if  (hinshi[0] == "名詞" ) and  (hinshi[0] not in exceptype) and  (hinshi[1] not in exceptype )and (hinshi[2] not in exceptype )and (word not in excepword) : #今回、除外名詞以外だったら、
                dictionary[-1] = dictionary[-1]+word   # 辞書の最後の単語にこの単語を結合
                prev_seq  +=1
                flag = 0
                
            elif (hinshi[1] == "接尾" ): #接尾辞だったら
                dictionary[-1] = dictionary[-1]+word   # とりあえず辞書の最後の単語にこの単語を結合して
                prev_seq  +=1
                flag = 1 #フラグを1に

            elif flag == 1: # 一個前が接尾辞で今度が名詞でなかったら、一個前の接尾辞で終わってる単語を辞書から削除
                dictionary.pop(-1)   #一個前の接尾辞で終わってる単語を辞書から削除
                prev_seq  =0
                flag = 0
            else:
                flag = 0
                if prev_seq  < sequence: 
                    dictionary.pop(-1) #一個前の単語を辞書から削除
                prev_seq = 0
        else:  # その前に名詞がつづいてなくて
            if  (hinshi[0] == "名詞" or hinshi[0] == "接頭辞")  and  hinshi[0] not in exceptype and  hinshi[1] not in exceptype and hinshi[2] not in exceptype and word not in excepword :
                dictionary.append(word)
                prev_seq += 1
                flag = 0

        node = node.next  
    return dictionary



# 単語ー文書行列の作成
docs = np.array(df['sentence'])

tfidf_vectorizer = TfidfVectorizer(
    tokenizer = tokenize_rm_setsubi, #上でつくった関数
    lowercase=True,
    max_df=5000,
    # smooth_idf = False,
    min_df=5)

tfidf_vectorizer.fit(docs)
terms= tfidf_vectorizer.get_feature_names()

count_vectrizer = CountVectorizer(
    vocabulary = terms,
    tokenizer = tokenize_rm_setsubi, #上でつくった関数
)

c_matrix = count_vectrizer.transform(docs)


/Users/WatanabeShingo/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [11]:
# np.save('c_matrix_allterm',c_matrix)
# sp.sparse.save_npz('c_matrix_allterm',c_matrix)
# terms_df = pd.DataFrame(terms)
# terms_df.to_csv('terms_df.csv', mode='w')

# c_matrix = np.load('c_matrix_allterm.npz', allow_pickle=True)
# terms_df = pd.read_csv('terms_df.csv')


In [6]:
c_matrix = sp.sparse.load_npz('c_matrix_allterm.npz')
terms_df = pd.read_csv('terms_df.csv')


In [7]:
print(type(c_matrix))

<class 'scipy.sparse.csr.csr_matrix'>


In [9]:
c_df = pd.DataFrame(terms_df.values, columns=['',"terms"] )
c_df["document_frequency"] =  np.array(c_matrix.astype(bool).sum(axis=0).reshape(-1,1))
c_df["max_term_frequency"]  = c_matrix.T.max(axis=1).toarray()
c_df["sumof_term_frequency"] = np.array(c_matrix.T.sum(axis=1))
c_df.sort_values('document_frequency', ascending=False) # 場合によってはコメントアウト

,,terms,document_frequency,max_term_frequency,sumof_term_frequency
41334,41334,断面形状,4776,53,12964
29925,29925,外径,3984,131,13722
16619,16619,作用効果,3923,22,7890
25506,25506,回動,3316,123,18753
36687,36687,所定間隔,3282,59,7635
...,...,...,...,...,...
32688,32688,岩石質材料,5,2,6
32685,32685,岩石繊維,5,3,7
32684,32684,岩石用ハンマー,5,1,5
32679,32679,岩盤，,5,1,5


In [ ]:
# c_df = pd.DataFrame(terms_df.values, columns=['',"terms"] )
# c_df["document_frequency"] =  np.array(c_matrix.astype(bool).sum(axis=0).reshape(-1,1))
# c_df["max_term_frequency"]  = c_matrix.T.max(axis=1).toarray()
# c_df["sumof_term_frequency"] = np.array(c_matrix.T.sum(axis=1))
# c_df.sort_values('document_frequency', ascending=False) # 場合によってはコメントアウト

In [10]:
re_obj = r'[↑↓°±℃≫≪≦∞【】！-／：-＠［-｀｛-～、-〜”’%]'
c_df = c_df.replace(re_obj, '', regex=True)

In [11]:
c_df.head()

,,terms,document_frequency,max_term_frequency,sumof_term_frequency
0,0,緯度,6,2,7
1,1,エルボ,9,10,24
2,2,カメラ,7,7,16
3,3,ピッチ,21,4,35
4,4,フック,12,3,19


In [16]:
c_df.to_csv("allterms.csv" )

In [14]:
print(c_matrix)

  (0, 12231)	1
  (0, 12238)	6
  (0, 12320)	1
  (0, 12978)	2
  (0, 15429)	3
  (0, 15431)	1
  (0, 21713)	2
  (0, 25305)	1
  (0, 27014)	1
  (0, 31413)	1
  (0, 32484)	1
  (0, 46212)	4
  (0, 52673)	1
  (0, 57266)	3
  (0, 57949)	1
  (0, 59774)	1
  (0, 64340)	1
  (0, 65876)	1
  (0, 68197)	8
  (0, 75655)	1
  (1, 3015)	4
  (1, 10748)	10
  (1, 13338)	6
  (1, 13348)	1
  (1, 13357)	2
  :	:
  (44563, 38765)	1
  (44563, 40430)	127
  (44563, 42402)	1
  (44563, 43688)	16
  (44563, 44318)	1
  (44563, 44347)	132
  (44563, 44366)	61
  (44563, 44378)	1
  (44563, 44410)	3
  (44563, 44435)	1
  (44563, 54710)	1
  (44563, 55594)	2
  (44563, 59231)	1
  (44563, 61131)	1
  (44563, 61132)	1
  (44563, 61858)	8
  (44563, 61953)	7
  (44563, 62010)	1
  (44563, 62253)	1
  (44563, 62254)	1
  (44563, 67002)	1
  (44563, 69285)	8
  (44563, 69826)	1
  (44563, 70134)	1
  (44563, 70244)	2


In [15]:
print(c_matrix.shape[0])
print(c_matrix.getrow(0).toarray()[0])


44564
[0 0 0 ... 0 0 0]


In [1]:
df.head()


NameError: name 'df' is not defined

In [19]:
#単語:文書内単語頻度 の辞書の列つくって、pandasにくっつける
def makeTermsTfDict(terms, vec):
    # dic = {key: val for key, val in zip(terms, vec)  if val } #dfが1より大きいtermの辞書つくる
    print(terms)
    dic = {key: val for key, val in zip(terms, vec)  }
    # dic2 = dict(sorted(dic.items(), key=lambda x:x[1],reverse=True)) #dfの値でソート
    return dic

df['term'] = pd.Series([makeTermsTfDict(terms, vec) for vec in [c_matrix.getrow(i).toarray()[0] for i in range(c_matrix.shape[0])]]) #countされたベクトルの各行を関数に入れていってシリーズに

NameError: name 'terms' is not defined

In [ ]:
df.to_excel('parcedPatents.xlsx', sheet_name='Sheet1', ascending=False)

In [ ]:
df.to_json('parcedPatents.json', ascending=False)